In [1]:
import re # regular expression
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
#converting a collection of text documents into a numerical representation
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
#Tokenization is the process of dividing a text into smaller units
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
#A layer which learns a position embedding for inputs sequences.
from keras.datasets import imdb

from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# A common solution for SSL certificate_verify_failed errors.

In [5]:
import pandas as pd
data = pd.read_csv(r"C:\Users\chetl\OneDrive\Documents\rnn_reviews.csv", encoding='latin1')

#data = pd.read_csv('Senti.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [6]:
data.head(10)

,text,sentiment
0,If you love Barbie this movie is for you.,postivie
1,Shrodingers reccomendation,postivie
2,This is much more than just a film about Barbie.,postivie
3,Not only did Mattel approve a PG-13 Barbie mov...,postivie
4,i still remember when i watched this in summer...,postivie
5,I thought it was brilliant.,postivie
6,The satire was very satisfying,postivie
7,im obsessed with the dream houses being comple...,postivie
8,I didnt expect this movie to turnout this goo...,postivie
9,This movie is a masterpiece.,postivie


In [7]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [8]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print('Shape of training samples:',X_train.shape,Y_train.shape)
print('Shape of testing samples:',X_test.shape,Y_test.shape)

Shape of training samples: (186, 45) (186, 2)
Shape of testing samples: (93, 45) (93, 2)


In [9]:
model = Sequential()
#model.add(Embedding(max_fatures, 128 ,input_length = X.shape[1], dropout=0.2))
model.add(Embedding(max_fatures, 128 ,input_length = X.shape[1]))
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 387842 (1.48 MB)
Trainable params: 387842 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [32]:
batch_size = 12
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
16/16 - 1s - loss: 5.1777e-04 - accuracy: 1.0000 - 1s/epoch - 65ms/step
Epoch 2/5
16/16 - 1s - loss: 3.7117e-04 - accuracy: 1.0000 - 1s/epoch - 64ms/step
Epoch 3/5
16/16 - 1s - loss: 3.1734e-04 - accuracy: 1.0000 - 1s/epoch - 64ms/step
Epoch 4/5
16/16 - 1s - loss: 2.9340e-04 - accuracy: 1.0000 - 1s/epoch - 64ms/step
Epoch 5/5
16/16 - 1s - loss: 4.4597e-04 - accuracy: 1.0000 - 1s/epoch - 65ms/step


In [33]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Accuracy: %.2f" % (acc))

8/8 - 0s - loss: 1.2373 - accuracy: 0.7312 - 250ms/epoch - 31ms/step
Score: 1.24
Accuracy: 0.73


In [36]:
text = 'I absolutely loved this movie! The storyline was captivating, the characters were well-developed, and the acting was superb. I was on the edge of my seat throughout the entire film. Highly recommend!'

tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['text']

tester['text'] = tester['text'].apply(lambda x: x.lower())
tester['text'] = tester['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
if(prediction[0][0]> 0.5):
    print("Negative")
else:
    print("Positive")

1/1 [==============================] - 0s 50ms/step
Prediction value: [1.8254044e-10 1.0000000e+00]
Positive
